In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI
from dotenv import load_dotenv
import os
import random
from storysim import StorySimulator
import pandas as pd

/homes/ngetach/miniconda/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')


In [4]:
# This example is the new way to use the OpenAI lib for python
client = OpenAI()

def prompt_gpt(prompt, model):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    
    return response.choices[0].message.content

In [43]:
# graph = {
#         "hole_1": ["hole_2", "field"],
#         "hole_2": ["hole_1", "hole_3"],
#         "hole_3": ["hole_2", "hole_4"],
#         "hole_4": ["hole_3", "field"],
#         "field": ["hole_1", "hole_4"]
#     }

# events = {
        
#         7:{"name": "cross_paths","actors": ["Alice", "Bob"], "location": "hole_2"}, 
#         8: {"name": "exclusive_random", "actors":["Alice", "Bob"], "stop": 10 },
#         13: {"name": "mislead", "actors":["Alice", "Bob"]}
#     }


# sim = StorySimulator(
#         people=["Alice", "Bob", "Charlie", "Danny"],
#         locations=["hole_1", "hole_2", "hole_3", "hole_4", "field"],
#         relation="jumps_in",
#         trial_seed=10,
#         params={'prompt': '3', 'type': 'cot'},
#         graph=graph,
#         events=events
#     )

### Experiment Setup

In [135]:
def mislead_experiment(actors, locs, g, mislead, length):
    poi = random.sample(actors, 2)
    loc = random.sample(locs, 1)
    second_loc = random.choice(g[loc[0]])
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc, "path_type": "same"}
    event_dict[11] = {"name":"move", "actor":poi[-1], "location": second_loc}
    event_dict[12] = {"name": "exclusive_random", "actors": poi, "stop": 12 + mislead}
    event_dict[12 + mislead] = {"name": "mislead", "actors": poi}
    event_dict[12 + mislead+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, second_loc, experiment_info

def spaced_mislead_experiment(actors, locs, g, mislead, length):
    event_dict = {}
    poi = random.sample(actors, 2)
    loc = random.sample(locs, 1)
    label = random.choice([l for l in g[loc[-1]] if l != loc[-1]])
    event_dict[15] = {"name": "cross_paths","actors": poi, "location": loc}
    event_dict[16] = {"name": "exclusive_random", "actors": poi, "stop": 17}
    event_dict[17] = {"name":"move", "actor":poi[-1],"location":label}
    event_dict[18] = {"name": "exclusive_random", "actors": poi, "stop": 18+mislead}
    event_dict[18+mislead] = {"name": "mislead", "actors": poi}
    event_dict[18 + mislead +1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, label, experiment_info

def number_of_moves_experiment(actors, locs, g, length):
    poi = random.sample(actors, 2)
    loc = random.sample(locs,1)
    num_moves = 0
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc, "path_type":"same"}
    prev = loc[0]
    movement = []
    for i in range(1,num_moves+1):
        new_loc = random.choice([l for l in g[prev] if l not in loc])
        movement.append(new_loc)
        event_dict[10+i] = {"name":"move", "actor":poi[-1], "location": new_loc}
        prev = new_loc
    #event_dict[10+num_moves+1] = {"name": "mislead", "actors": poi}
    label =  movement[0]
    event_dict[10+num_moves+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, label, experiment_info

def second_order_tom_experiment(actors, locs, g, length):
    poi = random.sample(actors, 3)
    loc_1 = random.sample(locs,1)
    loc_2 = random.sample(g[loc_1[0]], 1)
    loc_3 = random.sample([l for l in g[loc_2[0]] if l != loc_1[0]])
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc_1, "path_type":"same"}
    event_dict[16] = {"name": "cross_paths","actors": poi[1:], "location": loc_1, "path_type":"same"}
    event_dict[17] = {"name":"move", "actor":poi[-1],"location":loc_3}
    event_dict[18] = {"name": "exclusive_random", "actors": poi, "stop": length}
    

def cross_path_overlap(actors, locs, g, mislead, length, n):
    poi = random.sample(actors, n)
    loc = random.sample(locs, 1)
    second_loc = random.choice(g[loc[0]])
    event_dict = {}
    event_dict[15] = {"name": "cross_paths","actors": poi, "location": loc, "path_type": "same"}
    event_dict[16] = {"name":"move", "actor":poi[-1], "location": second_loc}
    event_dict[17] = {"name": "exclusive_random", "actors": poi, "stop": 17 + mislead}
    event_dict[17 + mislead] = {"name": "mislead", "actors": poi}
    event_dict[17 + mislead+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, second_loc, experiment_info

In [136]:
def find_k_unique_paths(graph, start, end, k):
    def dfs(node, path, visited, paths):
        if len(paths) >= k:  # Stop early if we found k paths
            return
        
        if node == end:  # If reached destination, store the path
            paths.append(list(path))
            return

        for neighbor in graph.get(node, []):  # Explore neighbors
            if neighbor not in visited:
                visited.add(neighbor)
                path.append(neighbor)

                dfs(neighbor, path, visited, paths)

                # Backtrack
                visited.remove(neighbor)
                path.pop()

    paths = []
    dfs(start, [start], {start}, paths)  # Start DFS
    return [len(p)-1 for p in paths[:k]],[p[1:] for p in paths[:k]]  # Return up to k paths

### Few shot setup

In [137]:
possible_people = ["Alice", "Bob", "Charlie", "Danny", "Edward", "Frank", "Georgia", "Hank", "Isaac", "Jake", "Kevin"]
num_people = 5

graph = { 
    "room_1": ["room_2", "the_hallway","room_5"],
    "room_2": ["room_1", "room_3","the_hallway"],
    "room_3": ["room_2", "room_4","the_hallway"],
    "room_4": ["room_3", "room_5","room_1"],
    "room_5": ["room_4", "room_1","room_2"],
    "the_hallway": ["room_1", "room_4","room_2"]
}

locations = list(graph.keys())

# Make fewshot examples:
event_dict = {}
fewshot_story_length = 25
n_fewshot = 3
mislead_fewshot = 3
fewshot_examples = []

random.seed(100)

for n in range(n_fewshot):
    
    event_dict, label, fewshot_experiment_dict = mislead_experiment(possible_people[:num_people], locations[:-1], graph, mislead_fewshot, fewshot_story_length)
    
    sim = StorySimulator(
        people=possible_people[:num_people],
        locations=list(graph.keys()),
        relation="enters",
        params={'prompt': '3', 'type': 'cot'},
        graph=graph,
        events=event_dict
    )
    res = sim.run_simulation(fewshot_story_length)
    story = sim.formal_to_story(res)
    fewshot_cot_label = f"The last place that {fewshot_experiment_dict['poi'][0]} and {fewshot_experiment_dict['poi'][1]} were in the same location was {fewshot_experiment_dict['cross path location']}. Then, {fewshot_experiment_dict['poi'][0]} observed {fewshot_experiment_dict['poi'][-1]} going into {label}. After this, {fewshot_experiment_dict['poi'][1]} changed locations, so {fewshot_experiment_dict['poi'][0]} no longer knows where {fewshot_experiment_dict['poi'][1]} is. Therefore, {fewshot_experiment_dict['poi'][0]} thinks that {fewshot_experiment_dict['poi'][1]} is in {label if 'hallway' not in label else 'the hallway'}."
    fewshot_label = label
    fewshot_examples.append((story, fewshot_experiment_dict['poi'], fewshot_cot_label))
    
print(fewshot_examples)

[('Charlie enters room_1. Edward enters room_2. Alice enters room_4. Alice enters room_5. Alice enters room_4. Bob enters room_4. Alice enters room_3. Charlie enters the_hallway. Edward enters room_3. Alice enters room_4. Danny enters room_4. Danny enters room_3. Charlie enters room_2. Charlie enters room_3. Alice enters room_1. Danny enters room_2. Alice enters room_2. Edward enters room_4. Alice enters the_hallway. Alice enters room_1. Alice enters room_2. Edward enters room_3. Charlie enters the_hallway. Charlie enters room_1. Edward enters the_hallway', ['Bob', 'Danny'], 'The last place that Bob and Danny were in the same location was room_4. Then, Bob observed Danny going into room_3. After this, Danny changed locations, so Bob no longer knows where Danny is. Therefore, Bob thinks that Danny is in room_3.'), ('Alice enters room_1. Danny enters room_2. Alice enters room_5. Danny enters room_3. Charlie enters room_2. Bob enters room_1. Edward enters room_4. Edward enters room_5. Edw

### Actual Experiment setup

In [138]:
df = pd.DataFrame({'Story':[], 'Label':[], 'P1':[], 'P2':[]})

In [139]:


possible_people = ["Alice", "Bob", "Charlie", "Danny", "Edward", "Frank", "Georgia", "Hank", "Isaac", "Jake", "Kevin"]
num_people = 7
# graph = {
#         "hole_1": ["hole_2", "field"],
#         "hole_2": ["hole_1", "hole_3"],
#         "hole_3": ["hole_2", "hole_4"],
#         "hole_4": ["hole_3", "field"],
#         "field": ["hole_1", "hole_4"]
#     }


# graph = { 
#     "hole_1": ["hole_2", "field", "hole_5", "hole_6", "hole_4"],
#     "hole_2": ["hole_1", "hole_3", "field", "hole_5", "hole_7"],
#     "hole_3": ["hole_2", "hole_4", "field", "hole_8", "hole_1"],
#     "hole_4": ["hole_3", "hole_5", "hole_1", "hole_9", "field"],
#     "hole_5": ["hole_4", "hole_1", "hole_2", "hole_10", "field"],
#     "hole_6": ["hole_1", "hole_7", "hole_9", "field"],
#     "hole_7": ["hole_6", "hole_2", "hole_8", "field"],
#     "hole_8": ["hole_7", "hole_3", "hole_9", "field"],
#     "hole_9": ["hole_8", "hole_4", "hole_6", "hole_10", "field"],
#     "hole_10": ["hole_9", "hole_5", "field", "hole_7"],
#     "field": ["hole_1", "hole_2", "hole_3", "hole_4", "hole_9", "hole_10"]
# }

graph = { 
    "room_1": ["room_2", "the_hallway","room_5"],
    "room_2": ["room_1", "room_3","the_hallway"],
    "room_3": ["room_2", "room_4","the_hallway"],
    "room_4": ["room_3", "room_5","room_1"],
    "room_5": ["room_4", "room_1","room_2"],
    "the_hallway": ["room_1", "room_4","room_2"]
}

locations = list(graph.keys())
story_length = 25
num_trials = 40
mislead_distance = 3

random.seed(25)

for _ in range(num_trials):
    event_dict, label, experiment_dict = cross_path_overlap(possible_people[:num_people], locations[:-1], graph, mislead_distance, story_length, 5)
   

    print(experiment_dict['poi'])
    sim = StorySimulator(
        people=possible_people[:num_people],
        locations=locations,
        relation="enters",
        params={'prompt': '3', 'type': 'cot'},
        graph=graph,
        events=event_dict
    )
    res = sim.run_simulation(story_length)
    
    story = sim.formal_to_story(res)

    d = {'Story':[], 'Label':[], 'P1':[], 'P2':[]}
    d['P1'] = ",".join(experiment_dict['poi'][:-1]) if len(experiment_dict['poi'][:-1]) > 1 else experiment_dict['poi'][:-1]
    #print(d["P1"])
    d['P2'] = experiment_dict['poi'][-1]
    d['Story'].append(story)
    # d['Label'].append(movement[0])
    d['Label'].append(label)
    df = pd.concat([df, pd.DataFrame(d)])

['Danny', 'Alice', 'Bob', 'Charlie', 'Georgia']
['Alice', 'Danny', 'Charlie', 'Bob', 'Georgia']
['Georgia', 'Alice', 'Charlie', 'Bob', 'Edward']
['Alice', 'Edward', 'Charlie', 'Georgia', 'Bob']
['Georgia', 'Edward', 'Charlie', 'Bob', 'Danny']
['Bob', 'Charlie', 'Frank', 'Danny', 'Edward']
['Bob', 'Edward', 'Frank', 'Georgia', 'Charlie']
['Danny', 'Alice', 'Georgia', 'Bob', 'Edward']
['Bob', 'Charlie', 'Alice', 'Danny', 'Edward']
['Frank', 'Danny', 'Georgia', 'Alice', 'Charlie']
['Edward', 'Charlie', 'Georgia', 'Alice', 'Bob']
['Charlie', 'Edward', 'Bob', 'Georgia', 'Frank']
['Charlie', 'Georgia', 'Alice', 'Edward', 'Danny']
['Edward', 'Charlie', 'Frank', 'Alice', 'Danny']
['Danny', 'Alice', 'Edward', 'Charlie', 'Georgia']
['Danny', 'Charlie', 'Bob', 'Edward', 'Georgia']
['Georgia', 'Charlie', 'Frank', 'Danny', 'Bob']
['Danny', 'Georgia', 'Edward', 'Charlie', 'Frank']
['Frank', 'Alice', 'Charlie', 'Georgia', 'Danny']
['Frank', 'Georgia', 'Danny', 'Edward', 'Bob']
['Frank', 'Danny', 'Geo

In [140]:
df.head()

,Story,Label,P1,P2
0,Edward enters room_2. Charlie enters room_4. B...,room_3,"Danny,Alice,Bob,Charlie",Georgia
0,Edward enters room_1. Charlie enters room_4. E...,room_5,"Alice,Danny,Charlie,Bob",Georgia
0,Danny enters room_1. Alice enters room_4. Bob ...,room_2,"Georgia,Alice,Charlie,Bob",Edward
0,Frank enters room_1. Frank enters the_hallway....,room_5,"Alice,Edward,Charlie,Georgia",Bob
0,Edward enters room_4. Bob enters room_4. Charl...,the_hallway,"Georgia,Edward,Charlie,Bob",Danny


In [141]:
for i ,row in df.iterrows():
    print("\n".join(row['Story'].split(".")))
    print('---')

Edward enters room_2
 Charlie enters room_4
 Bob enters room_4
 Frank enters room_2
 Frank enters the_hallway
 Edward enters the_hallway
 Edward enters room_2
 Alice enters room_4
 Frank enters room_4
 Frank enters room_3
 Edward enters the_hallway
 Danny enters room_4
 Frank enters room_2
 Edward enters room_4
 Edward enters room_1
 Georgia enters room_4
 Georgia enters room_3
 Edward enters room_5
 Edward enters room_2
 Edward enters room_1
 Georgia enters room_2
 Frank enters the_hallway
 Frank enters room_4
 Frank enters room_1
 Frank enters the_hallway
---
Edward enters room_1
 Charlie enters room_4
 Edward enters the_hallway
 Edward enters room_1
 Frank enters room_4
 Frank enters room_5
 Frank enters room_1
 Danny enters room_4
 Frank enters the_hallway
 Frank enters room_4
 Bob enters room_4
 Frank enters room_5
 Alice enters room_4
 Edward enters the_hallway
 Frank enters room_1
 Georgia enters room_4
 Georgia enters room_5
 Edward enters room_2
 Edward enters the_hallway
 Edw

In [142]:
fewshot_examples[0]

('Charlie enters room_1. Edward enters room_2. Alice enters room_4. Alice enters room_5. Alice enters room_4. Bob enters room_4. Alice enters room_3. Charlie enters the_hallway. Edward enters room_3. Alice enters room_4. Danny enters room_4. Danny enters room_3. Charlie enters room_2. Charlie enters room_3. Alice enters room_1. Danny enters room_2. Alice enters room_2. Edward enters room_4. Alice enters the_hallway. Alice enters room_1. Alice enters room_2. Edward enters room_3. Charlie enters the_hallway. Charlie enters room_1. Edward enters the_hallway',
 ['Bob', 'Danny'],
 'The last place that Bob and Danny were in the same location was room_4. Then, Bob observed Danny going into room_3. After this, Danny changed locations, so Bob no longer knows where Danny is. Therefore, Bob thinks that Danny is in room_3.')

### Run Model

In [143]:
intial_prompt = f"Read the following story and answer the question at the end. Note that all characters start in {sim.locations[-1].replace('_',' ')}. Characters in the same location can see where eachother go when someone leaves. If characters are in different locations, they cannot see eachother."
responses = []
fewshot = False
model_choice = "gpt-4"
for _ ,row in df.iterrows():
    p = row['P1'].split(',')
    formatted = []
    answer = ''
    if fewshot:
        #formatted = [f'{ex[0]}\nQ: Where does {ex[1][0]} think {ex[1][1]} thinks {ex[1][2]} is?\nA: {ex[2]}' for ex in fewshot_examples]
        formatted = [f'{ex[0]}\nQ: Where does {ex[1][0]} think {ex[1][1]} is?\nA: {ex[2]}' for ex in fewshot_examples]
        formatted = '\n'.join(formatted)
        prompt = f"{intial_prompt}\n{formatted}\n{row['Story']}\nQ: Where does {p[0]} think {row['P2']} is?\nA:"
        print(prompt)
        print('*****')
        answer = prompt_gpt(prompt, model_choice)
    else:
        prompt = f"{intial_prompt}\n{row['Story']}\nQ: Where does {p[0]} think {row['P2']} is?"
        print(prompt)
        print("=====")
        answer = prompt_gpt(prompt, model_choice)
    responses.append(answer)        
df['Responses'] = responses

Read the following story and answer the question at the end. Note that all characters start in the hallway. Characters in the same location can see where eachother go when someone leaves. If characters are in different locations, they cannot see eachother.
Edward enters room_2. Charlie enters room_4. Bob enters room_4. Frank enters room_2. Frank enters the_hallway. Edward enters the_hallway. Edward enters room_2. Alice enters room_4. Frank enters room_4. Frank enters room_3. Edward enters the_hallway. Danny enters room_4. Frank enters room_2. Edward enters room_4. Edward enters room_1. Georgia enters room_4. Georgia enters room_3. Edward enters room_5. Edward enters room_2. Edward enters room_1. Georgia enters room_2. Frank enters the_hallway. Frank enters room_4. Frank enters room_1. Frank enters the_hallway
Q: Where does Danny think Georgia is?
=====
Read the following story and answer the question at the end. Note that all characters start in the hallway. Characters in the same loca

In [144]:
for trial in range(num_trials):
    print(df['Responses'].iloc[trial], f", {df['Label'].iloc[trial]}")
    print('--')

Danny thinks Georgia is in room 4. , room_3
--
Alice thinks Georgia is in room 4. , room_5
--
Georgia thinks Edward is in room 3. , room_2
--
Alice thinks Bob is in room 1. , room_5
--
Georgia thinks Danny is in room_3. , the_hallway
--
Bob thinks Edward is in room 5. , room_5
--
Bob thinks Charlie is in room 5. , room_4
--
Danny thinks Edward is in room 2. , room_2
--
Bob thinks Edward is in room_3. , the_hallway
--
Frank thinks Charlie is in room 3. , room_4
--
Edward thinks Bob is in room 4. , room_3
--
Charlie thinks Frank is in room 2. , room_2
--
Charlie thinks Danny is in room 5. , room_1
--
Edward thinks Danny is in room 2. , the_hallway
--
Danny thinks Georgia is in room_1. , room_1
--
Danny thinks Georgia is in room_3. , room_2
--
Georgia thinks Bob is in room 3. , room_2
--
Danny thinks Frank is in room 2. , room_1
--
Frank thinks Danny is in room 5. , room_1
--
Frank thinks Bob is in room 2. , room_3
--
Frank thinks Edward is in room_2. , the_hallway
--
Alice thinks Frank i

### Compute Accuracy

In [145]:
def compute_score_unsure(label, response):
    base = label.split("_")[0] if not label.startswith(sim.locations[-1]) else label
    response = response.split("\n")[-1]
    if response.count(base) <= 1:
        return str(label in response or label.replace('_'," ") in response)
    elif 'Therefore,' in response:
        return str(label in response.split('Therefore,')[-1] or label in response.split('Therefore,')[-1])
    return f'{response}, {label}'

outs= df.apply(lambda x: compute_score_unsure(x['Label'], x['Responses']), axis=1)
known = [k for k in outs if k == 'True' or k == 'False']
unkown = [k for k in outs if k != 'True' and k != 'False']
print(sum([1 for k in known if k == 'True']))
print("\n--\n".join(unkown))

10



In [ ]:
#df.apply(lambda x: x['Label'] in x['Responses'], axis=1)
#df.to_csv("saved_results/gpt4_40_doubletom.csv", index=False) 
1/40

0.425

In [147]:
d = pd.read_csv('saved_results/gpt4_40_3moves_pilot.csv')
d.head()

,Story,Label,P1,P2,Responses
0,Bob jumps in hole_2. Charlie jumps in hole_1. ...,hole_3,Danny,Alice,Danny thinks Alice is in the field.
1,Charlie jumps in hole_1. Charlie jumps in hole...,hole_1,Alice,Bob,Alice thinks Bob is in the field.
2,Alice jumps in hole_4. Alice jumps in hole_1. ...,field,Bob,Charlie,Bob thinks Charlie is in hole_4.
3,Alice jumps in hole_1. Alice jumps in hole_3. ...,field,Bob,Danny,Bob thinks Danny is in hole_2.
4,Charlie jumps in hole_4. Danny jumps in hole_4...,hole_3,Bob,Alice,Bob thinks Alice is in the field.


In [148]:
print('\n'.join(d['Story'].to_list()[0].split('.')))
print(d['Label'].to_list()[0])
d['Pred'] = d.apply(lambda x: compute_score_unsure(x['Label'], x['Responses']), axis=1)

Bob jumps in hole_2
 Charlie jumps in hole_1
 Charlie jumps in hole_3
 Charlie jumps out to the field
 Bob jumps out to the field
 Bob jumps in hole_2
 Charlie jumps in hole_4
 Charlie jumps in hole_3
 Danny jumps in hole_2
 Bob jumps out to the field
 Alice jumps in hole_2
 Alice jumps in hole_3
 Alice jumps out to the field
 Bob jumps in hole_1
 Charlie jumps out to the field
 Bob jumps in hole_3
 Bob jumps out to the field
 Bob jumps in hole_2
 Bob jumps in hole_1
 Bob jumps out to the field
 Charlie jumps in hole_4
 Charlie jumps in hole_1
 Charlie jumps out to the field
 Bob jumps in hole_4
 Charlie jumps in hole_1
hole_3


In [149]:
wrong = d[d['Pred'] == 'False']
print('\n'.join(wrong['Story'].to_list()[10].split('.')))
print(wrong['Label'].to_list()[10])
print(wrong['P1'].to_list()[10])
print(wrong['P2'].to_list()[10])
print(wrong['Responses'].to_list()[10])



Danny jumps in hole_1
 Danny jumps in hole_2
 Danny jumps in hole_3
 Danny jumps in hole_2
 Bob jumps in hole_1
 Alice jumps in hole_2
 Alice jumps out to the field
 Alice jumps in hole_1
 Bob jumps in hole_3
 Charlie jumps in hole_1
 Charlie jumps in hole_3
 Charlie jumps in hole_4
 Charlie jumps in hole_1
 Danny jumps in hole_3
 Danny jumps out to the field
 Danny jumps in hole_2
 Danny jumps in hole_1
 Danny jumps in hole_3
 Danny jumps in hole_2
 Alice jumps out to the field
 Alice jumps in hole_2
 Danny jumps out to the field
 Danny jumps in hole_4
 Danny jumps in hole_3
 Danny jumps in hole_2
hole_4
Bob
Charlie
Bob thinks Charlie is in hole_1.
